In [1]:
from Bio.Seq import Seq
from Bio.SeqUtils import MeltingTemp
from Bio.SeqUtils import lcc, GC
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio import SeqIO,SeqRecord
from Bio.Alphabet import IUPAC

from sklearn.utils import shuffle
from joblib import dump, load
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas(ascii=True)
%matplotlib inline

In [12]:

def plot_align(s1,s2):
    score = len(s1)-hamming_distance(s1,s2)
    print(format_alignment(s1,s2, score, 0, len(s1)))
    

def align(s1,s2):
    score = len(s1)-hamming_distance(s1,s2)
    res = format_alignment(s1,s2, score, 0, len(s1))
    return res
    
    
    
# def ssDNA_generator(length = 40, random_state = 10):
#     np.random.seed(random_state)
#     ssDNA = ''.join(np.random.choice(['A','T','G','C'],size = length)) 
#     my_seq = Seq(ssDNA)
    
#     c1 =  (GC(my_seq) < 45)
#     c2 =  (GC(my_seq) > 55) 
#     while c1 | c2:
#         ssDNA = ''.join(np.random.choice(['A','T','G','C'],size = length)) 
#         my_seq = Seq(ssDNA)  
#         if  45 < GC(my_seq) < 55:
#             break
#     return my_seq

def ssDNA_generator(length = 40, random_state = 10):
    np.random.seed(random_state)
    ssDNA = ''.join(np.random.choice(['A','T','G','C'],size = length)) 
    my_seq = Seq(ssDNA)
    

    return my_seq


def hamming_distance(s1, s2):
    """Return the Hamming distance between equal-length sequences"""
    if len(s1) != len(s2):
        raise ValueError("Undefined for sequences of unequal length")
    return sum(el1 != el2 for el1, el2 in zip(s1, s2))


def isRepeat(seq, r = 4):
    
    h = False
    for i in range(len(seq)):
        sub = seq[i:i+r]    
        if len(sub) == r:            
            if len(set(sub)) == 1:
                h  = True
                break        
    return h
    
    
def mutation(seq, i, ss = ['A','T','C','G']):
    """ Site-directed mutagenesis """
    m_seq = seq.tomutable()
    m_seq[i] = np.random.choice(ss, 1)[0]   
    return m_seq.toseq()


def mutation2(seq, sites, choice = ['T','G','A','C']):
    """ Site-directed mutagenesis """
    m_seq = seq.tomutable()
    for i in sites:
        x = choice.copy()
        m_seq[i] = np.random.choice(x, 1)[0]
    return m_seq.toseq()




# def coevolution(s1, s2, hd1 = 40, hd2 = 35, r = 4):
#     """coevolution slection algorithm  
#     parameters
#     ----------------------
#     s1: sequence 1
#     s2: sequence 2
#     hd: threshold of hamming distance
#     r:  number of repeat bases
#     """
    
#     #conditions
#     c1 =  hamming_distance(s1,s2) < hd1
#     c2 =  hamming_distance(s1,s2.reverse_complement()) < hd1
#     c3 =  hamming_distance(s1, s1.reverse_complement()) < hd2
#     c4 =  hamming_distance(s2, s2.reverse_complement())  < hd2
    
#     c5 = isRepeat(s1, r)
#     c6 = isRepeat(s2, r)
    
#     c7 =  (GC(s1) < 45) 
#     c8 =  (GC(s2) < 45)
#     c9 =  (GC(s1) > 55) 
#     c10 = (GC(s2) > 55)
    
#     k = int(r/2)
    
    
#     c = c1 | c2 | c3 | c4 | c5 | c6 |  c7 |  c8 | c9 | c10
#     while  c:
        
#         site1 = []
#         site2 = []
#         for i in range(len(s1)):
            
            
#             if c7:
#                 if (s1[i] == 'A') | (s1[i] == 'T'):
#                     s1 = mutation(s1, i, ['C', 'G'])
                    
#             if c8:
#                 if (s2[i] == 'A') | (s2[i] == 'T'):
#                     s2 = mutation(s2, i, ['C', 'G'])
                    
#             if c9:
#                 if (s1[i] == 'G') | (s1[i] == 'C'):
#                     s1 = mutation(s1, i, ['A', 'T'])  
                    
#             if c10:
#                 if (s2[i] == 'G') | (s2[i] == 'C'):
#                     s2 = mutation(s2, i, ['A', 'T'])  
                    
                    
#             if c1:
#                 if s1[i] == s2[i]:
#                     s2 = mutation(s2, i)
#             if c2:
#                 if s1[i] == s2.reverse_complement()[i]:
#                     s1 = mutation(s1, i)
                    
#             # self mutation                    
#             if c3:
#                 if s1[i] == s1.reverse_complement()[i]:
#                     s1 = mutation(s1, i)
                    
#             # self mutation
#             if c4:
#                 if s2[i] == s2.reverse_complement()[i]:
#                     s2 = mutation(s2, i)
                    
#             if c5:
#                 sub = s1[i:i+r]    
#                 if len(sub) == r:            
#                     if len(set(sub)) == 1:
#                         s1 = mutation(s1, i+k)
                        
#             if c6:
#                 sub = s2[i:i+r]    
#                 if len(sub) == r:            
#                     if len(set(sub)) == 1:
#                         s2 = mutation(s2, i+k)
      

#             #print(c1,c2,c3,c4,c5,c6, c7, c8,c9,c10)   
                    
#         c1 =  hamming_distance(s1,s2) < hd1
#         c2 =  hamming_distance(s1,s2.reverse_complement()) < hd1
#         c3 =  hamming_distance(s1, s1.reverse_complement()) < hd2
#         c4 =  hamming_distance(s2, s2.reverse_complement())  < hd2
#         c5 = isRepeat(s1, r)
#         c6 = isRepeat(s2, r)   
    
#         c7 =  (GC(s1) < 45) 
#         c8 =  (GC(s2) < 45)
#         c9 =  (GC(s1) > 55) 
#         c10 = (GC(s2) > 55)

#         c = c1 | c2 | c3 | c4 | c5 | c6 |  c7 |  c8 | c9 | c10
        
#     #print(c)
#     return s1,s2


def coevolution(s1, s2, hd1 = 40, hd2 = 35, r = 4):
    """coevolution slection algorithm  
    parameters
    ----------------------
    s1: sequence 1
    s2: sequence 2
    hd: threshold of hamming distance
    r:  number of repeat bases
    """
    
    #conditions
    c1 =  hamming_distance(s1,s2) < hd1
    c7 =  (GC(s1) < 45) 
    c8 =  (GC(s2) < 45)
    c9 =  (GC(s1) > 55) 
    c10 = (GC(s2) > 55)
    
    k = int(r/2)
    
    
    c = c1 |  c7 |  c8 | c9 | c10
    while  c:

        site2 = []
        site1 = []
        for i in range(len(s1)):              
            if c1:
                if s1[i] == s2[i]:
                    site2.append(i)
            if c7 | c9:
                site1.append(i)
            if c8 | c10:
                site2.append(i)
  

        s1 = mutation2(s1, site1)
        s2 = mutation2(s2, site2)

                    
        c1 =  hamming_distance(s1,s2) < hd1
       
        c7 =  (GC(s1) < 45) 
        c8 =  (GC(s2) < 45)
        c9 =  (GC(s1) > 55) 
        c10 = (GC(s2) > 55)

        c = c1 | c7 |  c8 | c9 | c10
        
    #print(c)
    return s1,s2



def metrics(s1,s2):
    """A with B, A with Brc, A with Arc, B with Brc"""
    m1 =  hamming_distance(s1,s2) #similar
    m2 =  hamming_distance(s1,s2.reverse_complement())  #complement
    m3 =  hamming_distance(s1, s1.reverse_complement()) #self
    m4 =  hamming_distance(s2, s2.reverse_complement()) #self 
    return {'A_B':m1, 'A_Brc':m2, 'A_Arc':m3, 'B_Brc':m4}

def evaluate_products(products):
    performances = []
    for i in range(len(products)):
        for j in range(len(products)):   
            if i == j:
                continue
                
            s1 = products[i]
            s2 = products[j]
            performances.append(metrics(s1, s2))
    df = pd.DataFrame(performances)
    return df

def train(total = 21, products = []):
    
    s = ssDNA_generator()
    if not products:
        products = [s for i in range(total)]

    performances = []
    for i in range(len(products)):
        for j in range(len(products)):   
            if i == j:
                continue
                
            s1 = products[i]
            s2 = products[j]
            performances.append([i,j, metrics(s1, s2)])
            s1,s2 = coevolution(s1, s2)
            products[i] = s1
            products[j] = s2
            
    return performances, products


def save(products, name = "example.fasta"):
    records = []
    for i, seq in enumerate(products):
        records.append(SeqRecord.SeqRecord(seq, id = str(i+1), description = str(i+1)))
    with open(name, "w") as output_handle:
        SeqIO.write(records, output_handle, "fasta")
        
def save_products_align(products):
    for i, s1 in enumerate(products):
        a = []
        for j, s2 in enumerate(products):
            if s1 == s2:
                continue
            res = align(s1,s2)
            words = '%s vs %s:\n' % (i,j) + res + '\n\n'
            a.append(words)
        with open('./data/%s.txt' % str(i).zfill(4), 'w') as f:
            f.writelines(a)

In [13]:
s1 = ssDNA_generator(random_state=10)
s2 = ssDNA_generator(random_state = 10)

coevolution(s1,s2)

(Seq('GCGAGTAAAGTGCGTCAGGACATCTATGCATACTCGACCT'),
 Seq('TTTTCGTTTAATACCGCAATATGGGCATGGGCTGGACAAC'))

In [15]:
performances, products_new = train(products = products)

In [21]:
df = pd.DataFrame(performances)


In [24]:
epochs = 100
distance_thr = 0
res1 = []
products = []
for i in tqdm(range(epochs), ascii = True):
    
    products = shuffle(products)
    performances, products_new = train(products = products)
    df = pd.DataFrame(performances)
    df = df.join(df[2].apply(pd.Series))
    ss = round(df[['A_B']].min().mean(), 3)
    
    while distance_thr >= ss:
        products_new = shuffle(products_new)
        performances, products_new = train(products = products_new)
        df = pd.DataFrame(performances)
        df = df.join(df[2].apply(pd.Series))
        ss = round(df[['A_B']].min().mean(), 3)
    distance_thr = ss
    products = products_new
    res1.append(ss)
    print(ss)
    save(products, name='./data/%s.fasta' % str(i).zfill(4))
    dump(products, './data/%s_%s.prod' % (str(i).zfill(4), ss))


  1%|1         | 1/100 [00:02<03:26,  2.09s/it]

16.0


KeyboardInterrupt: 

In [27]:
        df = pd.DataFrame(performances)
        df = df.join(df[2].apply(pd.Series))

In [31]:
df.sort_values('A_B')

,0,1,2,A_B,A_Brc,A_Arc,B_Brc
350,17,10,"{'A_B': 15, 'A_Brc': 32, 'A_Arc': 30, 'B_Brc':...",15,32,30,30
371,18,11,"{'A_B': 16, 'A_Brc': 24, 'A_Arc': 24, 'B_Brc':...",16,24,24,20
244,12,4,"{'A_B': 16, 'A_Brc': 35, 'A_Arc': 36, 'B_Brc':...",16,35,36,34
316,15,17,"{'A_B': 17, 'A_Brc': 30, 'A_Arc': 28, 'B_Brc':...",17,30,28,30
246,12,6,"{'A_B': 17, 'A_Brc': 35, 'A_Arc': 36, 'B_Brc':...",17,35,36,32
...,...,...,...,...,...,...,...
314,15,14,"{'A_B': 40, 'A_Brc': 28, 'A_Arc': 28, 'B_Brc':...",40,28,28,32
335,16,15,"{'A_B': 40, 'A_Brc': 32, 'A_Arc': 26, 'B_Brc':...",40,32,26,28
356,17,16,"{'A_B': 40, 'A_Brc': 27, 'A_Arc': 30, 'B_Brc':...",40,27,30,26
398,19,18,"{'A_B': 40, 'A_Brc': 28, 'A_Arc': 34, 'B_Brc':...",40,28,34,24


In [ ]:
#     save(products, name='./data/%s.fasta' % str(i).zfill(4))
#     dump(products, './data/%s_%s.prod' % (str(i).zfill(4), ss))

In [ ]:
df['lcc'] = df['seq'].progress_apply(lambda x:lcc.lcc_simp(x))
df= pd.DataFrame(products, columns = ['seq'])
df['GC_rate'] = df.seq.progress_apply(lambda x:GC(x))
    

In [11]:
df

,A_B,A_Brc,A_Arc,B_Brc
0,27,28,32,28
1,30,29,32,34
2,28,31,32,32
3,30,30,32,28
4,22,31,32,34
...,...,...,...,...
415,26,28,28,34
416,30,28,28,28
417,27,27,28,26
418,25,24,28,28
